Example of how to use HSNE Python parser for reading and analysing HSNE hierarchies

In [1]:
from HSNE_parser import read_HSNE_binary
import pandas as pd
import numpy as np

Read the hierarchy from a HSNE binary file and print progress:

In [2]:
hsne = read_HSNE_binary(filename="./sample_data/mnis_aoi.bin", verbose=True)

Number of scales 3
Start reading first scale of size 1000
Done reading first scale..

Next scale: 1
Scale size: 236
Reading transmatrix..
Reading landmarks of scale to original data..
Reading landmarks to previous scale..
Reading landmark weights..
Reading previous scale to current scale..
Reading area of influence..
Time spent converting 1D to sparse: 0.065597

Next scale: 2
Scale size: 29
Reading transmatrix..
Reading landmarks of scale to original data..
Reading landmarks to previous scale..
Reading landmark weights..
Reading previous scale to current scale..
Reading area of influence..
Time spent converting 1D to sparse: 0.014401
Total time spent parsing hierarchy and building objects: 0.180012


In [16]:
# How many scales do I have?
hsne.num_scales

3

1 data scale and 2 landmarkscales

In [3]:
for scale in hsne:
    print(scale)

HSNE datascale 0 with 1000 datapoints
HSNE subscale 1 with 236 datapoints
HSNE subscale 2 with 29 datapoints


Intra scale similarities are stored in scale.tmatrix, a sparse matrix os size scalesize * scalesize:

In [4]:
#scale 0 is just a k=30 knn graph
hsne[0].tmatrix

<1000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 30000 stored elements in COOrdinate format>

In [5]:
hsne[1].tmatrix

<236x236 sparse matrix of type '<class 'numpy.float64'>'
	with 11445 stored elements in COOrdinate format>

Inter scale similarities are stored in scale.area_of_influence (datascale does not have an area of inluence). A sparse matrix of size previous scale * current scale:

In [6]:
hsne[1].area_of_influence

<1000x236 sparse matrix of type '<class 'numpy.float64'>'
	with 37321 stored elements in Compressed Sparse Column format>

Landmark information is stored in each scale (not the datascale) of the hsne object:

In [26]:
# Which original datapoint is each landmark?
# Landmark #0 in this scale was originally datapoint #0
hsne[1].lm_to_original[:20]

[1, 3, 8, 14, 17, 23, 26, 39, 40, 42, 43, 57, 72, 74, 95, 96, 97, 98, 102, 103]

In [25]:
# Which landmark in this scale was which landmark  in the previous scale?
# Landmark #0 in this scale was landmark #5 in the previous scale
hsne[2].lm_to_previous

[5,
 6,
 13,
 16,
 17,
 30,
 31,
 35,
 42,
 43,
 61,
 75,
 79,
 104,
 106,
 114,
 116,
 117,
 118,
 136,
 139,
 140,
 156,
 182,
 192,
 194,
 197,
 214,
 233]

### important !
For each landmark on the previous scale, which point on this scale best represents it?

So original datapoint #0 is best represented by landmark #192 on scale 1.
Also, landmark #0 on scale 1 is best represented by landmark #12 on scale 2:


In [31]:

print(hsne[1].best_representatives[0:10])
print(hsne[2].best_representatives[0:10])

[192 153 106   5 231 229  37  31 104 225]
[12  0 13 13  3  0 22 18 13 22]


In [32]:
hsne[2].best_representatives

array([12,  0, 13, 13,  3,  0, 22, 18, 13, 22, 22, 28, 13,  4, 12, 22,  3,
        2, 13, 23, 13, 13, 28, 20, 12, 13, 24, 18, 13, 28, 15,  8, 16, 28,
       12, 20,  4, 13, 24, 24,  3, 27,  6, 19,  1, 23, 21,  6, 23,  3,  1,
        6, 28, 22, 15, 23,  3,  1, 25,  9,  4, 17,  1, 28, 13,  6,  3, 28,
       23, 12, 23, 23, 13, 18, 22, 27, 21, 28,  8, 12,  6, 28, 26, 23, 21,
        3, 11, 21, 20, 23,  0, 13, 13, 18,  9, 28, 21, 23, 28, 28, 23, 28,
       18, 27, 13, 12, 28,  4, 28, 23,  1, 28, 24, 28, 18, 28, 20, 10, 15,
        8, 22, 27,  3, 25, 18, 13, 13, 21, 24, 23,  0, 13, 23,  0, 25, 24,
       20, 24, 25, 16, 21,  7,  3, 13,  0, 20,  1,  4, 23, 13, 22, 22, 13,
       12,  5, 23, 23, 12, 13, 13, 24, 23,  3, 12,  6, 23, 28, 13,  8, 15,
       23,  3, 28, 22,  6,  5, 12,  0, 23,  1,  3, 25, 22, 27, 13, 12, 27,
        5,  8,  3, 24, 23, 24, 20, 27,  6, 20, 18, 22,  3,  5,  3, 25, 25,
       24, 13, 23,  5, 24, 23, 12,  6, 23,  8, 25, 23,  5, 22, 27,  1,  5,
       24, 27,  3, 12, 28

The previous cell does not show which original datapoint is best represented by points on scale 2, these best representatives can be propagated through the scales with the get_datascale_mappings() function of the full hsne object.

The argument passed to get_datascale_mappings() is the scale for which you would like the mapping, scale 1 is simply the same as hsne[1].best_representatives but as a dictionary. For scale 2 the mapping gets propagated down through scale 1's best representatives.


In [33]:
hsne.get_datascale_mappings(1)

{0: 192,
 1: 153,
 2: 106,
 3: 5,
 4: 231,
 5: 229,
 6: 37,
 7: 31,
 8: 104,
 9: 225,
 10: 188,
 11: 137,
 12: 17,
 13: 216,
 14: 18,
 15: 70,
 16: 50,
 17: 162,
 18: 197,
 19: 29,
 20: 139,
 21: 185,
 22: 77,
 23: 1,
 24: 125,
 25: 42,
 26: 6,
 27: 42,
 28: 182,
 29: 129,
 30: 17,
 31: 171,
 32: 197,
 33: 129,
 34: 34,
 35: 137,
 36: 197,
 37: 14,
 38: 156,
 39: 118,
 40: 205,
 41: 16,
 42: 15,
 43: 198,
 44: 17,
 45: 99,
 46: 142,
 47: 38,
 48: 13,
 49: 42,
 50: 17,
 51: 163,
 52: 156,
 53: 147,
 54: 156,
 55: 16,
 56: 117,
 57: 108,
 58: 136,
 59: 214,
 60: 139,
 61: 5,
 62: 197,
 63: 157,
 64: 233,
 65: 38,
 66: 30,
 67: 58,
 68: 117,
 69: 34,
 70: 167,
 71: 70,
 72: 92,
 73: 216,
 74: 17,
 75: 14,
 76: 194,
 77: 194,
 78: 75,
 79: 209,
 80: 17,
 81: 14,
 82: 127,
 83: 216,
 84: 156,
 85: 122,
 86: 17,
 87: 11,
 88: 117,
 89: 233,
 90: 114,
 91: 165,
 92: 139,
 93: 30,
 94: 16,
 95: 34,
 96: 156,
 97: 199,
 98: 13,
 99: 75,
 100: 20,
 101: 132,
 102: 3,
 103: 132,
 104: 91,
 105: 2

Original datapoint #0 is best represented by landmark #24 on scale 2:

In [35]:
hsne.get_datascale_mappings(2)

{0: 24,
 1: 12,
 2: 28,
 3: 0,
 4: 1,
 5: 5,
 6: 13,
 7: 8,
 8: 13,
 9: 28,
 10: 8,
 11: 24,
 12: 2,
 13: 5,
 14: 13,
 15: 23,
 16: 1,
 17: 3,
 18: 18,
 19: 28,
 20: 16,
 21: 12,
 22: 28,
 23: 0,
 24: 13,
 25: 6,
 26: 22,
 27: 6,
 28: 22,
 29: 23,
 30: 2,
 31: 3,
 32: 18,
 33: 23,
 34: 12,
 35: 24,
 36: 18,
 37: 12,
 38: 23,
 39: 15,
 40: 13,
 41: 3,
 42: 22,
 43: 22,
 44: 2,
 45: 28,
 46: 3,
 47: 24,
 48: 4,
 49: 6,
 50: 2,
 51: 12,
 52: 23,
 53: 4,
 54: 23,
 55: 3,
 56: 10,
 57: 28,
 58: 20,
 59: 25,
 60: 16,
 61: 0,
 62: 18,
 63: 12,
 64: 28,
 65: 24,
 66: 15,
 67: 25,
 68: 10,
 69: 12,
 70: 13,
 71: 23,
 72: 13,
 73: 5,
 74: 2,
 75: 12,
 76: 27,
 77: 27,
 78: 27,
 79: 23,
 80: 2,
 81: 12,
 82: 21,
 83: 5,
 84: 23,
 85: 3,
 86: 2,
 87: 28,
 88: 10,
 89: 28,
 90: 18,
 91: 23,
 92: 16,
 93: 15,
 94: 3,
 95: 12,
 96: 23,
 97: 3,
 98: 4,
 99: 27,
 100: 13,
 101: 23,
 102: 13,
 103: 23,
 104: 13,
 105: 25,
 106: 18,
 107: 8,
 108: 10,
 109: 13,
 110: 16,
 111: 8,
 112: 13,
 113: 25,
 114